In [10]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-vpngbjrx
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-vpngbjrx
  Resolved https://github.com/ibm-granite-community/utils to commit 5d67648927240b208a164d2466f0dc77200450e5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [11]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    model="ibm-granite/granite-20b-code-instruct-8k",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_length":300, "temperature":0.2},
)

In [12]:
def zeroshot_prompt(context, question, book_titles):
    """
    Creates a zero-shot prompt for the model, where the model acts as a seasoned programmer.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question or task for the model to perform
    - book_titles: list, list of book titles to include in the prompt
    Returns:
    - str, the formatted prompt
    """

    titles = ", ".join(book_titles)
    prompt = f"""
    You are an experienced programmer with 15 years of experience writing full-stack applications.
    Your task is to generate code for an application using the context provided in the task: {context}.
    Begin by creating components based on the user question: {question}.
    Include a description of elements provided in {titles}.
    Ensure that you generate only Python code for a Jupyter Notebook using ipywidgets UI components.
    Do not output anything but the code. The output code should not exceed 300 tokens.
    """
    return prompt


In [13]:
def get_answer_using_zeroshot(context, question, book_titles):
    """
    Generates the response from the model based on a zero-shot prompt.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question for the model to answer
    - book_titles: list, list of book titles to include in the prompt

    Returns:
    - str, the generated result from the model
    """
    prompt = zeroshot_prompt(context, question, book_titles)
    result = model.invoke(prompt)

    return result

In [15]:
context = "Design and develop an online bookstore UI components with minimalistic theme."
question = "Create the landing page for users visiting my bookstore. The landing page should display a header `Reader's Online Store`, a welcome message `Welcome to Reader's Verse` along with a catalog of books titles as title. Vertically align the sections."
book_titles = ["The Great Gatsby", "Pride and Prejudice", "The Hobbit", "The Lord of the Rings", "Animal Farm", "Brave New World"]

# Generate and display the UI code for the landing page
result = get_answer_using_zeroshot(context, question, book_titles)
print(f"Generated Code:\n{result}")

ReplicateError: ReplicateError Details:
title: Version disabled
status: 422
detail: This deployment is currently disabled:
- This deployment has been disabled in its settings.

In [8]:
import ipywidgets as widgets
# Define the header
header = widgets.HTML("Reader's Online Store")
# Define the welcome message
welcome_message = widgets.HTML("Welcome to Reader's Verse")
# Define the book catalog
book_catalog = widgets.HTML("Catalog of Books:")
# Define the book titles and descriptions
book_titles = ["The Great Gatsby", "Pride and Prejudice", "The Hobbit", "The Lord of the Rings", "Animal Farm", "Brave New World"]
book_descriptions = ["F. Scott Fitzgerald's classic novel set in the Roaring Twenties, following the decadent lifestyle of Jay Gatsby and his wife Daisy Buchanan.",
 "The story follows the romantic exploits of the Bennet sisters, Mary and Elizabeth, as they navigate the social norms of the Regency era in England.",
 "The Hobbit is a fantasy novel by J. R. R. Tolkien. It was published in 1937 and is the first book of The Lord of the Rings.",
 "The Lord of the Rings is an epic high-fantasy novel written by J. R. R. Tolkien.",
 "Animal Farm is a novella by George Orwell. It was published in 1945 and is a political satire of the Soviet Union.",
 "Brave New World is a 1932 novel by Aldous Huxley, published by Secker & Warburg as The Doors of Perception, and is regarded by many critics as one of the greatest novels of the 20th century."]
# Create a list of book descriptions
book_desc_list = []
for i in range(len(book_titles)):
 book_desc_list.append(widgets.HTML(f"{book_titles[i]}: {book_descriptions[i]}"))
# Create a vertical box layout for the landing page
landing_page = widgets.VBox([header, welcome_message, book_catalog] + book_desc_list)
# Display the landing page
landing_page